<a href="https://colab.research.google.com/github/majiddaeinejad/quera-ai-bootcamp-divar/blob/hypothesis_test/Hypothesis%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Load Data**

In [24]:
import pandas as pd
import gdown

file_id = "1fYGb-n114IGWZDcBKE0jzc3BWSLCcsVQ"

url = f"https://drive.google.com/uc?id={file_id}"

output = "Divar.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv(output)

print(df.head())

Downloading...
From (original): https://drive.google.com/uc?id=1fYGb-n114IGWZDcBKE0jzc3BWSLCcsVQ
From (redirected): https://drive.google.com/uc?id=1fYGb-n114IGWZDcBKE0jzc3BWSLCcsVQ&confirm=t&uuid=cf88af1f-026c-442f-8c40-849ed6e523fa
To: /content/Divar.csv
100%|██████████| 794M/794M [00:13<00:00, 60.4MB/s]
/tmp/ipython-input-1679883135.py:11: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output)


   Unnamed: 0         cat2_slug       cat3_slug city_slug neighborhood_slug  \
0           0    temporary-rent           villa     karaj         mehrshahr   
1           1  residential-sell  apartment-sell    tehran           gholhak   
2           2  residential-rent  apartment-rent    tehran             tohid   
3           3   commercial-rent     office-rent    tehran          elahiyeh   
4           4  residential-sell  apartment-sell   mashhad          emamreza   

      created_at_month    user_type  \
0  2024-08-01 00:00:00  مشاور املاک   
1  2024-05-01 00:00:00  مشاور املاک   
2  2024-10-01 00:00:00          NaN   
3  2024-06-01 00:00:00          NaN   
4  2024-05-01 00:00:00  مشاور املاک   

                                         description  \
0  ۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...   
1  دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...   
2                                    تخلیه پایان ماه   
3  فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...   
4  هلدینگ 

In [25]:
file_id = "1eX9an3tSypvrFn_fiPReYHTca5GpJaJG"

url = f"https://drive.google.com/uc?id={file_id}"

df_cities = pd.read_csv(url)
df_cities

,نام شهر,دسته‌بندی
0,karaj,کلان‌شهر
1,tehran,کلان‌شهر
2,mashhad,کلان‌شهر
3,ahvaz,کلان‌شهر
4,kermanshah,کلان‌شهر
...,...,...
235,baneh,شهر کوچک
236,ahmadsar-gourab,شهر کوچک
237,saqqez,شهر کوچک
238,mianeh,شهر کوچک


فرض اول

In [26]:
from scipy import stats

# ادغام کردن دیتا برای فهمیدن دسته بندی هر شهر به کلان شهر یا شهر کوچک
df_city = df_cities.rename(columns={"نام شهر": "city_slug", "دسته‌بندی": "city_type"})
df_one = pd.merge(df, df_city, on="city_slug", how="inner").copy()

df_one = df_one[df_one["cat2_slug"].isin(["residential-sell", "residential-rent"])]

#حذف مقادیر گم شده در مساحت خانه
df_one = df_one.dropna(subset=["building_size"])

megacities = df_one[df_one["city_type"] == "کلان‌شهر"]["building_size"]
small_cities = df_one[df_one["city_type"] == "شهر کوچک"]["building_size"]


#انجام آزمون نرمال بودن توزیع به جهت اینکه ببینیم از چه آزمون فرضی استفاده کنیم
#میبینیم اگر دیتا خیلی زیاد بود فقط 5000 عدد از داده رو برمیداریم
p1 = stats.shapiro(megacities.sample(min(5000, len(megacities))))[1]
p2 = stats.shapiro(small_cities.sample(min(5000, len(small_cities))))[1]
print( p1)
print( p2)



if p1 > 0.05 and p2 > 0.05:
    # t-test یک‌طرفه؟
    stat, p_val = stats.ttest_ind(megacities, small_cities, alternative="less")
    test_used = "t-test"
else:
    # داده نرمال نیست
    stat, p_val = stats.mannwhitneyu(megacities, small_cities, alternative="less")
    test_used = "Mann–Whitney U"



print(f"\nآزمون مورد استفاده: {test_used}")
print(f"آماره آزمون: {stat:.4f}")
print(f"p-value: {p_val:.4f}")

if p_val < 0.05:
    print("تایید میشه")
else:
    print("شواهد کافی برای تایید فرضیه وجود ندارد")

1.4335479752759908e-95
2.567832208707325e-95

آزمون مورد استفاده: Mann–Whitney U
آماره آزمون: 70410392554.0000
p-value: 0.0000
تایید میشه


فرض دوم

In [27]:
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu

df2 = df_final.copy()


#عبارت "قبل از ۱۳۷۰" به سال ۱۳۶۹ تبدیل شده تا بتوان مقایسه عددی انجام داد.
df2.loc[df2.construction_year=='قبل از ۱۳۷۰', 'construction_year'] = 1369
#تغییر مقادیر به فلوت
df2.construction_year = df2.construction_year.astype('float')
#حذف ردیف مقادیر گمشده
df2=df2.dropna(subset=["building_size","construction_year"])



old_houses = df2[df2["construction_year"] < 1396]["building_size"].dropna()
new_houses = df2[df2["construction_year"] >= 1396]["building_size"].dropna()




sample_size = 5000 if len(old_houses) > 5000 else len(old_houses)
shapiro_old = shapiro(old_houses.sample(sample_size, random_state=1))
sample_size = 5000 if len(new_houses) > 5000 else len(new_houses)
shapiro_new = shapiro(new_houses.sample(sample_size, random_state=1))

print("Shapiro-Wilk Old Houses:", shapiro_old)
print("Shapiro-Wilk New Houses:", shapiro_new)

#آزمون برابری واریانس‌ها
levene_test = levene(old_houses, new_houses)
print("Levene Test:", levene_test)

alpha = 0.05
#انجام آزمون بر اساس تست هایی که انجام دادیم
if shapiro_old.pvalue >= alpha and shapiro_new.pvalue >= alpha and levene_test.pvalue >= alpha:
    t_stat, p_val = ttest_ind(old_houses, new_houses, alternative='greater')
    test_name = "Independent Samples t-test (one-sided: old > new)"
else:
    t_stat, p_val = mannwhitneyu(old_houses, new_houses, alternative='greater')
    test_name = "Mann–Whitney U Test (one-sided: old > new)"

print(f"\nSelected Test: {test_name}")
print(f"Test Statistic: {t_stat:.4f}")
print(f"P-Value: {p_val:.4g}")

if p_val < alpha:
    print(" نتیجه: خانه‌های قدیمی به طور معناداری مساحت بیشتری دارند")
else:
    print(" شواهد کافی برای تفاوت معنادار پیدا نشد")

Shapiro-Wilk Old Houses: ShapiroResult(statistic=np.float64(0.007572024033949631), pvalue=np.float64(1.1320100460106781e-95))
Shapiro-Wilk New Houses: ShapiroResult(statistic=np.float64(0.014094434858509364), pvalue=np.float64(1.6212064518214097e-95))
Levene Test: LeveneResult(statistic=np.float64(3.87662843404578), pvalue=np.float64(0.04896312769920616))

Selected Test: Mann–Whitney U Test (one-sided: old > new)
Test Statistic: 64810415402.5000
P-Value: 1
 شواهد کافی برای تفاوت معنادار پیدا نشد


فرض سوم

In [28]:
df3 = df_final.copy()
df3 = df_final[df_final["cat2_slug"] == "commercial-sell"]
#حذف ردیف مقادیر گمشده در ستون های مورد نیاز
df3 = df.dropna(subset=["has_business_deed", "price_value"])
#تفکیک داده ها بر اساس فرض
with_deed = df3[df3["has_business_deed"] == True]["price_value"]
without_deed = df3[df3["has_business_deed"] == False]["price_value"]
#تست توزیع نرمال
sample_size = min(5000, len(with_deed))
shapiro_with = shapiro(with_deed.sample(sample_size, random_state=1))

sample_size = min(5000, len(without_deed))
shapiro_without = shapiro(without_deed.sample(sample_size, random_state=1))

print("Shapiro Test (With Deed):", shapiro_with)
print("Shapiro Test (Without Deed):", shapiro_without)
#تست واریانش
levene_test = levene(with_deed, without_deed)
print("Levene Test:", levene_test)
#انتخاب ازمون فرض بر اساس تست ها
alpha = 0.05
if (shapiro_with.pvalue >= alpha) and (shapiro_without.pvalue >= alpha) and (levene_test.pvalue >= alpha):
    stat, p_value = ttest_ind(with_deed, without_deed, alternative="two-sided")
    test_used = "Independent Samples t-test (two-sided)"
else:
    stat, p_value = mannwhitneyu(with_deed, without_deed, alternative="two-sided")
    test_used = "Mann–Whitney U Test (two-sided)"

print(f"\nSelected Test: {test_used}")
print(f"Test Statistic: {stat:.4f}")
print(f"P-Value: {p_value:.4g}")

if p_value < alpha:
    print("  تفاوت معناداری بین میانگین قیمت فروش دو گروه وجود دارد.")
else:
    print("  شواهد کافی برای تفاوت معنادار پیدا نشد.")

Shapiro Test (With Deed): ShapiroResult(statistic=np.float64(0.0208015803204028), pvalue=np.float64(2.3506486442099992e-95))
Shapiro Test (Without Deed): ShapiroResult(statistic=np.float64(0.025812807086604495), pvalue=np.float64(3.107212764858161e-95))
Levene Test: LeveneResult(statistic=np.float64(0.01023547521276335), pvalue=np.float64(0.9194158035161516))

Selected Test: Mann–Whitney U Test (two-sided)
Test Statistic: 113571227.0000
P-Value: 3.045e-60
  تفاوت معناداری بین میانگین قیمت فروش دو گروه وجود دارد.


فرض چهارم

In [29]:
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu

df4 = df_final.copy()

df4 = df4[df4["cat2_slug"] == "residential-sell"]
#اضافه کردن یک ستون که ببینیم یک خاه لاکچری هست یا نه
#درصورتی که اصلا حتی یکی ازین ویژگی هارا نداشته باشد و سایر ویژگی ها مشخص نباشد و داده گمشده باشد آنرا اصلا داخل تست و آزمون فرض نمیاریم، چون ممکن هست اون داده ای که نیستش درست باشه و خانه جزو لاکچری به حساب بیاد
luxury_features = ["has_pool", "has_sauna", "has_jacuzzi", "has_barbecue"]
has_true = df4[luxury_features].eq(True).any(axis=1)

all_false = df4[luxury_features].eq(False).all(axis=1)
df4["is_luxury"] = pd.NA
df4.loc[has_true, "is_luxury"] = True
df4.loc[all_false, "is_luxury"] = False
#حذف داده های ناموجود
df4 = df4.dropna(subset=["price_value"])




luxury_prices = df4[df4["is_luxury"] == True]["price_value"]
non_luxury_prices = df4[df4["is_luxury"] == False]["price_value"]
#تست توزیع نرمال
sample_size = min(5000, len(luxury_prices))
shapiro_lux = shapiro(luxury_prices.sample(sample_size, random_state=1))

sample_size = min(5000, len(non_luxury_prices))
shapiro_nonlux = shapiro(non_luxury_prices.sample(sample_size, random_state=1))

print("Shapiro Test (Luxury):", shapiro_lux)
print("Shapiro Test (Non-Luxury):", shapiro_nonlux)
#تست واریانس
levene_test = levene(luxury_prices, non_luxury_prices)
print("Levene Test:", levene_test)
#انتخاب ازمون فرض بر اساس تست ها
alpha = 0.05
if (shapiro_lux.pvalue >= alpha) and (shapiro_nonlux.pvalue >= alpha) and (levene_test.pvalue >= alpha):
    stat, p_val = ttest_ind(luxury_prices, non_luxury_prices, alternative="two-sided")
    test_name = "Independent Samples t-test (two-sided)"
else:
    stat, p_val = mannwhitneyu(luxury_prices, non_luxury_prices, alternative="two-sided")
    test_name = "Mann–Whitney U Test (two-sided)"

print(f"\nSelected Test: {test_name}")
print(f"Test Statistic: {stat:.4f}")
print(f"P-Value: {p_val:.4g}")

if p_val < alpha:
    print(" نتیجه: بین قیمت‌های گروه لاکچری و غیرلاکچری تفاوت معناداری وجود دارد.")
else:
    print(" نتیجه: شواهد کافی برای تفاوت معنادار پیدا نشد.")


Shapiro Test (Luxury): ShapiroResult(statistic=np.float64(0.010594924612544299), pvalue=np.float64(1.33669810864339e-95))
Shapiro Test (Non-Luxury): ShapiroResult(statistic=np.float64(0.024199875438459584), pvalue=np.float64(2.839927052409882e-95))
Levene Test: LeveneResult(statistic=np.float64(0.13816970731708678), pvalue=np.float64(0.7101109540524805))

Selected Test: Mann–Whitney U Test (two-sided)
Test Statistic: 53465981.0000
P-Value: 9.449e-18
 نتیجه: بین قیمت‌های گروه لاکچری و غیرلاکچری تفاوت معناداری وجود دارد.
